In [1]:
import re
from vnpy.trader.constant import Exchange
from vnpy.app.cta_strategy.base import EXCHANGE_SYMBOL_DICT
from datetime import datetime, timedelta

In [36]:
fromDate = datetime.strptime('2012-6-10', '%Y-%m-%d')
toDate = datetime.strptime('2017-12-01', '%Y-%m-%d')

count = 0
while fromDate.date() != toDate.date():
    count += 1
    fromDate = fromDate + timedelta(1)

print(count)

2000


In [16]:
# 当前时间戳
import time
a = int(time.time())
print(a)

1563864223


In [59]:
# 时间戳转时间
timeStamp = 1512136800
date = datetime.fromtimestamp(timeStamp)
print(date)

2017-12-01 22:00:00


In [23]:
# 时间转时间戳
a = '2019-07-20 08:00:00'
b = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
print(b.timestamp())

1563580800.0


In [1]:
import requests
URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges'
response = requests.get(URL_EXCHANGES).json()
data = response['Data']
print(data.keys())

In [62]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from vnpy.app.cta_strategy.base import TICK_DB_NAME, MINUTE_DB_NAME, DAILY_DB_NAME, MinuteDataBaseName, HourDataBaseName
from datetime import datetime
from vnpy.trader.object import BarData, TickData

In [73]:
# ====== Tick ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
symbol_list = ['BTCUSD', 'ETHUSD', 'EOSUSD']
for symbol in symbol_list:
    cl = dbTick[symbol]
    cl.create_index('datetime')         # 添加索引
    start = datetime.strptime('2019-11-18 07:59:50', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-19 08:00:10', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = cl.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
        tick.__dict__ = dic
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}')
        count += 1
        if count >= 1:
            break

vt_symbol:BTCUSD.BYBIT	2020-03-25 07:52:46.091000	BTCUSD	6754.5
vt_symbol:ETHUSD.BYBIT	2020-03-25 08:01:59.786000	ETHUSD	137.85
vt_symbol:EOSUSD.BYBIT	2020-03-25 08:01:59.204000	EOSUSD	2.336


In [7]:
# ====== Minute ======
client = MongoClient('localhost', 27017)
db = client[MINUTE_DB_NAME]
collection = db['BTCUSD.BYBIT']
#collection = db['ZN2006']
start = datetime.strptime('2020-3-24 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2020-3-25 00:00:00', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = collection.find().sort('datetime', DESCENDING)
count = 0
for dic in cursor:
    print(dic['datetime'], dic['open_price'])
    count += 1
    if count >= 6:
        break

2020-05-28 14:36:00 9152.0
2020-05-28 14:35:00 9149.5
2020-05-28 14:34:00 9149.5
2020-05-28 14:33:00 9151.0
2020-05-28 14:32:00 9165.5
2020-05-28 14:31:00 9155.5


In [63]:
# ====== Daily ======
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
symbol_list = ['BTCUSD.BYBIT', 'ETHUSD.BYBIT', 'EOSUSD.BYBIT']
#symbol_list = ['BTC.USDT.OKEX', 'ETH.USDT.OKEX', 'EOS.USDT.OKEX']
#symbol_list = ['OKEX/BTC.USDT', 'OKEX/ETH.USDT', 'OKEX/EOS.USDT']
#symbol_list = ['Poloniex/BTC.USDT', 'Poloniex/ETH.USDT', 'Poloniex/EOS.USDT']
for symbol in symbol_list:
    collection = db[symbol]
    start = datetime.strptime('2019-11-27 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-27 00:06:00', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        symbol = dic['symbol']
        the_datetime = dic['datetime']
        open_price = dic['open_price']
        high_price = dic['high_price']
        low_price = dic['low_price']
        close_price = dic['close_price']
        print(f'{symbol}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
        count += 1
        break

BTCUSD.BYBIT	2020-05-27 08:00:00	8836.0	9230.0	8813.0	9202.0
ETHUSD.BYBIT	2020-05-27 08:00:00	201.0	208.5	200.75	208.35
EOSUSD.BYBIT	2020-05-27 08:00:00	2.504	2.574	2.457	2.568


In [5]:
symbol_list = ['RB2005', 'HC2005', 'SM2005', 'J2005', 'ZC2005', 'TA2005',
              'RB99', 'HC99', 'SM99', 'J99', 'ZC99', 'TA99']
symbol_list = ['BTCUSD.BYBIT']
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
for symbol in symbol_list:
    collection = db[symbol]
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        count += 1
        if count == 1:
            symbol = dic['symbol']
            exchange = dic['exchange']
            the_datetime = dic['datetime']
            open_price = dic['open_price']
            high_price = dic['high_price']
            low_price = dic['low_price']
            close_price = dic['close_price']
            print(f'{symbol}\t{exchange}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
            break
        
    print('\n')

BTCUSD.BYBIT	None	2019-12-24 08:00:00	7310.0	7429.0	7155.5	7254.0




In [59]:
# ====== 检查Tick质量 ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
cl = dbTick['BTCUSD']
cl.create_index('datetime')         # 添加索引
start = datetime.strptime('2019-12-20 07:59:50', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-12-30 08:00:10', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = cl.find(flt).sort('datetime')
count = 0
for dic in cursor:
    tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
    tick.__dict__ = dic
    if tick.last_price == 0:
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}\t{tick.ask_price_1}\t{tick.bid_price_1}')
    count += 1
print(count)

IndentationError: expected an indented block (<ipython-input-59-025e50492adc>, line 18)

In [61]:
from datetime import datetime
import csv
import os
content = {'datetime':datetime.now(), 'symbol':'rb2010', 'tick_price':1000, 'a':1, 'b':2}
file_path = 'test_test.csv'

field_names = list(content.keys())
file_exist = os.path.exists(file_path)
with open(file_path, 'a') as f:
    writer = csv.DictWriter(f, fieldnames=field_names)
    if not file_exist:
        writer.writeheader()
    # 写入csv文件
    writer.writerow(content)